In [4]:
import pandas as pd

data = pd.read_csv('./data/web_search_metrics.csv')
selected_search_prompt = 2
data = data[data["prompt"] == selected_search_prompt]
data

,queries,num_message,stage,prompt,model,query_counts,answer_similarity
80,['книги для расширения кругозора и вдохновения...,0,web_search,2,gemma-2-9b-it-Q8_0_L,2,0.770142
81,"['сколько книг читать в год для саморазвития',...",1,web_search,2,gemma-2-9b-it-Q8_0_L,2,0.812713
82,['топ 5 упражнений для утренней зарядки видео'...,2,web_search,2,gemma-2-9b-it-Q8_0_L,2,0.752011
83,"['история успеха известного спортсмена', 'как ...",3,web_search,2,gemma-2-9b-it-Q8_0_L,2,0.656919
84,['как правильно обрезать плодовые деревья весн...,4,web_search,2,gemma-2-9b-it-Q8_0_L,2,0.563283
...,...,...,...,...,...,...,...
355,"['Как создать эффективную команду для работы',...",35,web_search,2,suzume-llama-3-8b-Q8_0,2,0.707698
356,['Как окружающая среда влияет на моё физическо...,36,web_search,2,suzume-llama-3-8b-Q8_0,2,0.747172
357,"['Как развить навыки общения?', 'Методы для ул...",37,web_search,2,suzume-llama-3-8b-Q8_0,2,0.807445
358,"['Как путешествия влияют на личность?', 'Путеш...",38,web_search,2,suzume-llama-3-8b-Q8_0,2,0.719724


In [5]:
queries = []
for row in data["queries"].to_list():
    queries.extend(eval(row))

In [6]:
len(set([query.lower() for query in queries]))

226

In [9]:
urls = pd.read_csv('./data/urls.csv')
urls = pd.concat([urls[["query", "url1"]].rename(columns={"url1": "url"}), urls[["query", "url2"]].rename(columns={"url2": "url"})], axis=0)
urls

,query,url
0,книги для расширения кругозора и вдохновения -...,https://www.mann-ivanov-ferber.ru/catalog/tag/...
1,топ 5-7 книг для саморазвития и поиска вдохнов...,https://skills-akademy.ru/top-5-knig-dlya-samo...
2,сколько книг читать в год для саморазвития -cy...,https://sportzania.ru/about/publikatsii/chto-b...
3,как развить привычку читать и выбрать интересн...,https://vk.com/@pokoleniye_m-privychka-chitat-...
4,топ 5 упражнений для утренней зарядки видео -c...,https://alphamed.by/utrennyaya-zaryadka-kak-pr...
...,...,...
235,Методы для улучшения коммуникативных навыков -...,https://sky.pro/media/kak-razvit-kommunikativn...
236,Как путешествия влияют на личность? -cyberleni...,https://today.travel/blog/blogs/kak-vliyaut-pu...
237,Путешествия: как они меняют нас? -cyberleninka...,https://www.tutu.ru/geo/journal/raznoe/article...
238,Как развить самодисциплину для достижения целе...,https://ast-academy.ru/blog/razvitie_samodisci...


In [10]:
import requests
from bs4 import BeautifulSoup

def extract_information(url):
    try:
        response = requests.get(url, timeout=3)
    except:
        return {"error": "connections error"}
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.title.string.replace("\xa0", " ") if soup.title else "No title found"
        paragraphs = [p.text.replace("\xa0", ' ') for p in soup.find_all("p")]
        
        return {
            "title": title,
            "paragraphs": paragraphs
        }
    else:
        return {"error": f"Failed to retrieve the page. Status code: {response.status_code}"}

In [11]:
from tqdm import tqdm

info = []
pbar = tqdm(urls.iterrows(), total=len(urls))
for _, row in pbar:
    pbar.set_description(row.query)
    try:
        info.append(extract_information(row.url))
    except:
        info.append(None)

Практические советы для повышения уровня самодисциплины -cyberleninka -youtube -видео -pdf: 100%|█| 480/480 [05:13<00:0


In [22]:
urls['data'] = info

In [24]:
urls.to_csv('./data/page_information.csv', index=False)